In [ ]:
juan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import snowflake.connector
import requests
import io

In [ ]:
conn = snowflake.connector.connect(user='esteban.correa@RAPPI.COM', 
                                   authenticator='externalbrowser', 
                                   account='hg51401', 
                                   warehouse="RP_PERSONALUSER_WH",
                                   database="FIVETRAN")

In [ ]:
url='http://redash.rappi.com/api/queries/90464/results.csv?api_key=DEh28AiEeQ5z3f2Ppi6GITbECaSWfkBPyGxpI21D'
urlData = requests.get(url).content
inventario = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
inventario.head(2)

In [ ]:
#Cambio de nombre de columnas
inventario = inventario.rename(columns={"CRUCE":"cruce", "WAREHOUSE_ID":"warehouse_id", "WAREHOUSE_NAME":"warehouse_name", 
                                      "STOREREFERENCEID":"storereferenceid", "EAN":"ean", "STOCK":"stock"})
inventario.head(5)

In [ ]:
productos_exito = """SELECT * 
FROM FIVETRAN.CPGS_TURBO_DS_PUBLIC.CO_FILL_RATE_EXITO_PRODUCTO"""

In [ ]:
df_plu = pd.read_sql(productos_exito, conn)

In [ ]:
productos_exito = df_plu.rename(columns={"VIVO_ID":"storereferenceid", "PLU_PADRE":"plu_padre", "PLU_HIJO":"plu_hijo",
                                                  "Factor":"Factor"})
productos_exito.head(2)

In [ ]:
productos_exito = productos_exito[~productos_exito['storereferenceid'].isin([429540])]

In [ ]:
#Quitando de la base por problemas en la base
base_quitar = []
productos_exito = productos_exito[~productos_exito['storereferenceid'].isin(base_quitar)]

In [ ]:
tienda = """select f.*, w.name as warehouse_name, w.city as ciudad
from (select r.warehouse_id as sync_wh, r.external_id as warehouseid, e.external_id as dependencia
from fivetran.co_amysql_turbo_emergency_order_turbo_sync.warehouse_integration as r
left join fivetran.co_amysql_turbo_emergency_order_turbo_sync.warehouse_integration as e
on e.warehouse_id = r.warehouse_id
where r.integration_name in ('vivo')
and r._fivetran_deleted = 'FALSE'
and e.integration_name in ('exito')
and len(e.external_id) = 4
order by r.external_id desc) as f
left join fivetran.co_amysql_turbo_emergency_order_turbo_sync.warehouse as w
on f.sync_wh = w.id;"""

In [ ]:
tiendas = pd.read_sql(tienda, conn)
tiendas.head(2)

In [ ]:
tiendas = tiendas.rename(columns={"SYNC_WH":"sync_wh","WAREHOUSEID":"warehouseid", "DEPENDENCIA":"dependencia", 
                                  "WAREHOUSE_NAME":"tienda", "CIUDAD":"ciudad"})
tiendas.head(2)

In [ ]:
tiendas = tiendas.loc[:,["warehouseid", "dependencia"]]

In [ ]:
tiendas

In [ ]:
productos_exito = productos_exito.astype({"storereferenceid":"int"})  
stock_plu = pd.merge(left=inventario, right=productos_exito, how='inner', left_on="storereferenceid", right_on="storereferenceid")
stock_plu.head(2)

In [ ]:
tiendas.info()

In [ ]:
tiendas['warehouseid'] = tiendas['warehouseid'].astype(int)
stock_plu_tienda = pd.merge(left=stock_plu, right=tiendas, how='inner', left_on="warehouse_id", right_on="warehouseid")
stock_plu_tienda.head(2)

In [ ]:
len(stock_plu_tienda)

In [ ]:
stock_plu_tienda.info()

In [ ]:
stock_plu_tienda.head(5)

In [ ]:
stock_plu_tienda.sort_values(by = ["FACTOR"])

In [ ]:
#Cambiar tipo de dato de factor
stock_plu_tienda["FACTOR"]=stock_plu_tienda["FACTOR"].str.replace(",",".")
stock_plu_tienda = stock_plu_tienda.astype({"FACTOR":"float"})

In [ ]:
#Crear nueva columna que divida el factor
stock_plu_tienda["stock_final"] = stock_plu_tienda["stock"]/stock_plu_tienda["FACTOR"]
stock_plu_tienda.head(2)

In [ ]:
#Creación del archivo para manejo interno
stock_plu_tienda.to_excel("Informacion_stock.xlsx", index=False, sheet_name="Stock")

In [ ]:
stock_plu_tienda[stock_plu_tienda.plu_padre.isin(['3128771'])&stock_plu_tienda.dependencia.isin(['4842'])]

In [ ]:
stock_plu_tienda[stock_plu_tienda.plu_padre.isin(['3128771'])]

In [ ]:
stock_plu_tienda_2 = stock_plu_tienda.loc[:,["warehouseid", "warehouse_name", "plu_padre", 
                                           "dependencia", "stock_final"]]
stock_plu_tienda_2.head(2)

In [ ]:
#Agrupar información por plu padre
stock_plu_tienda_final = stock_plu_tienda_2.groupby(by=["warehouseid", "warehouse_name", "dependencia", 
                                                      "plu_padre"]).sum() 
stock_plu_tienda_final.head(2)

In [ ]:
stock_plu_tienda_final.reset_index(inplace=True)
stock_plu_tienda_final.head(2)

In [ ]:
stock_plu_tienda_final.to_excel("Inventario_exito.xlsx", index=False, sheet_name='Inventario')

In [ ]:
stock_plu_tienda_final.head(5)

In [ ]:
stock_plu_tienda_final[stock_plu_tienda_final.plu_padre.isin(['3128771'])&stock_plu_tienda_final.dependencia.isin(['4842'])]

In [ ]:
stock_plu_tienda_final[stock_plu_tienda_final.warehouseid.isin(['321', '322'])]

In [ ]:
stock_plu_tienda_final['stock_final'].sum(axis=0)

In [ ]:
import smtplib, getpass, os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from datetime import datetime
from email import encoders

user = """esteban.correa@rappi.com"""
password = """yexw ulvz yakj ggpc"""

fecha = datetime.today().strftime('%Y-%m-%d')

suma_productos = "{:.2f}".format(stock_plu_tienda_final['stock_final'].sum())

#Ajustes del mail
remitente = """Rappi Turbo Supply"""
destinatario = ['johan.mayorga@rappi.com','santiago.martin@rappi.com', 'donny.maury@rappi.com','marcel.kempe@rappi.com', 'olgonzalez@grupo-exito.com','daniel.montana@rappi.com', 'gmejia@grupo-exito.com', 'evenegas@grupo-exito.com', 'cesar.salcedo@rappi.com', 'dvillamila@grupo-exito.com', 'icbedoya@grupo-exito.com']
asunto = f"""Inventario EXITO {fecha}, suma productos: {suma_productos}"""
mensaje = """Hola team, <br><br> Enviamos inventario para el día de hoy.<br><br> ¡Gracias!"""

#Host y puerto SMTP de Gmail
server = smtplib.SMTP('smtp.gmail.com', 587)

#protocolo de cifrado de datos
server.starttls()

#Credenciales
server.login(user, password)

#Muestra la depuración de la operación de envío 1=true
server.set_debuglevel(1)

msg = MIMEMultipart()
msg['Subject'] = asunto
msg['From'] = remitente
msg['To'] = ", ".join(destinatario)


mensaje = MIMEText(mensaje, 'html')
msg.attach(mensaje)

part = MIMEBase('application', "octet-stream")
part.set_payload(open("Inventario_exito.xlsx", "rb").read())
encoders.encode_base64(part)

part.add_header('Content-Disposition', 'attachment; filename="Inventario_exito.xlsx"')
msg.attach(part)

#Enviar mail
server.sendmail(remitente, destinatario, msg.as_string())

#Cerrar la conexión
server.quit()

In [ ]:
user = """esteban.correa@rappi.com"""
password = """yexw ulvz yakj ggpc"""

fecha = datetime.today().strftime('%Y-%m-%d')

#Ajustes del mail
remitente1 = """Rappi Turbo Supply"""
destinatario1 = ['johan.mayorga@rappi.com','santiago.martin@rappi.com', 'donny.maury@rappi.com','marcel.kempe@rappi.com', 'olgonzalez@grupo-exito.com', 'daniel.montana@rappi.com', 'gmejia@grupo-exito.com', 'evenegas@grupo-exito.com', 'cesar.salcedo@rappi.com', 'dvillamila@grupo-exito.com', 'icbedoya@grupo-exito.com']
asunto1 = f"""Stock Turbo {fecha}"""
mensaje1 = f"""Hola, <br><br> Adjuntamos stock a la fecha {fecha} para cada vivoid.<br><br> ¡Gracias!"""

#Host y puerto SMTP de Gmail
server1 = smtplib.SMTP('smtp.gmail.com', 587)

#protocolo de cifrado de datos
server1.starttls()

#Credenciales
server1.login(user, password)

#Muestra la depuración de la operación de envío 1=true
server1.set_debuglevel(1)

msg1 = MIMEMultipart()
msg1['Subject'] = asunto1
msg1['From'] = remitente1
msg1['To'] = ", ".join(destinatario1)


mensaje1 = MIMEText(mensaje1, 'html')
msg1.attach(mensaje1)

part1 = MIMEBase('application', "octet-stream")
part1.set_payload(open("Informacion_stock.xlsx", "rb").read())
encoders.encode_base64(part1)

part1.add_header('Content-Disposition', 'attachment; filename="Informacion_stock.xlsx"')
msg1.attach(part1)

#Enviar mail
server1.sendmail(remitente1, destinatario1, msg1.as_string())

#Cerrar la conexión
server1.quit()

In [ ]:
conn.close()